<a href="https://colab.research.google.com/github/BlackHole3344/Deep-learning/blob/main/Unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers , models

In [ ]:
# Convolution block
# def conv(input , num_filters):
#   x = layers.Conv2D(num_filters , 3 , padding = "same")(input)
#   x = layers.BatchNormalization()(x)
#   x = layers.Activation("relu")(x)

#   x = layers.Conv2D(num_filters , 3 , padding = "same")(x)
#   x = layers.BatchNormalization()(x)
#   x = layers.Activation("relu")(x)

#   return x

In [ ]:
# x = Input([])

In [ ]:
def d_conv(inputs , filters):
  x = layers.Conv2D(filters , 3 , padding = "same")(inputs)
  x = layers.Conv2D(filters , 3 , padding = "same")(x)
  return x


In [ ]:
def downsample(input , num_filters ) :
  x = d_conv(input , num_filters)
  y = layers.MaxPool2D(pool_size = (2,2) )(x)
  return x , y



In [ ]:
def upsample(input , skip_connection , filters) :
  up = layers.UpSampling2D((2,2))(input)
  merge = layers.Concatenate([up , skip_connection])
  conv = d_conv(merge , filters)
  return conv


In [ ]:
# Building Unet
def unet(input_shape = (256 , 256 , 3) , no_classes = 1 ):
  input = layers.Input(input_shape)
  '''Down Sampling '''
  s1 , p1 = downsample(input , 32)
  s2 , p2 = downsample(p1 , 64)
  s3 , p3 = downsample(p2 , 128)
  s4 , p4 = downsample(p3 , 256)
  s5 , p5 = downsample(p4 , 512)

  # bottom
  b = d_conv(p4 , 1024)

  '''Upsampling'''
  u1 = upsample(b , s5 , 512)
  u2 = upsample(u1 , s4 , 256)
  u3 = upsample(u2 , s3 , 128 )
  u4 = upsample(u3 , s2 , 64)
  u5 = upsample(u4 , s1 , 32)

  out = layers.Conv2D(no_classes , 1 , activation = "sigmoid")(u5)

  model = tf.keras.Model(inputs = input , outputs = out )
  return model

In [ ]:
model = unet()
model.compile(optimizer = "Adam" , loss = 'binary_crossentropy' , metrics = ["accuracy"])


TypeError: Inputs to a layer should be tensors. Got '<keras.src.layers.merging.concatenate.Concatenate object at 0x7dfa0ae511b0>' (of type <class 'keras.src.layers.merging.concatenate.Concatenate'>) as input for layer 'conv2d_25'.